<a href="https://colab.research.google.com/github/ebtesam25/hackCU2021/blob/main/facial-recog/Facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-up


First, go to runtime->change runtime type -> and select GPU to improve the speed of training

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


Mount google drive to enable file browsing. Else, the data saved in your files will be deleted when the runtime is over

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'projects/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

In [9]:
!git clone https://github.com/ebtesam25/hackCU2021.git

Cloning into 'hackCU2021'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 274 (delta 0), reused 1 (delta 0), pack-reused 270
Receiving objects: 100% (274/274), 148.62 MiB | 33.46 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Checking out files: 100% (106/106), done.


In [10]:
#run cell everytime to pull update from repo
%cd
%cd /content
%cd hackCU2021
!git pull
%cd facial-recog

/root
/content
/content/hackCU2021
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/ebtesam25/hackCU2021
   34fad97..99586da  main       -> origin/main
Updating 34fad97..99586da
Fast-forward
 facial-recog/liveVideoFrameRead.py | 4 +++-
 1 file changed, 3 insertions(+), 1 deletion(-)
/content/hackCU2021/facial-recog


In [11]:
path_notebooks = '/content/hackCU2021/facial-recog/'
#os.listdir('/content/hackCU2021/notebooks/') #lists directory
#MODEL_DIR_PATH = path_notebooks + "models/" #path to models
#EXAMPLES_PATH = path_notebooks + "audio/" #path to audio files
#os.listdir(EXAMPLES_PATH) #lists directory
from config import root_dir
from config import base_dir
os.listdir(root_dir) #lists directory

['Facial_recognition.ipynb',
 'videoFrameRead.py',
 'export.pkl',
 '__pycache__',
 'hackCU2021',
 'projects',
 'haarcascade_frontalface_default.xml',
 'liveVideoFrameRead.py',
 'shape_predictor_68_face_landmarks.dat',
 'config.py']

In [13]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [14]:
!python liveVideoFrameRead.py

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute o